# Shared Memory Parallelism with OpenMP

During this exercise we will parallelize the stencil program from day 1 using OpenMP. The goal is to apply the OpenMP concepts that have been discussed in the lecture. If everything goes well, at the end of this exercise you will have a parallel version of the diffusion operator.

So let's start!

In [ ]:
import os
os.environ["OMP_NUM_THREADS"] = "72"
os.environ["OMP_PROC_BIND"] = "close"
os.environ["OMP_PLACES"] = "cores"

## Performance Baseline

In the first step we will see how fast our code performs and what the straightforward insertion of compiler directives (pragmas) can do.

In [ ]:
import timeit

import matplotlib
import matplotlib.pyplot as plt
import numpy as np

We start by compiling the Fortran stencil2d base code (this is the code with inlining you have worked on previously).

In [ ]:
%%bash
make clean

In [ ]:
%%bash
make clean
make VERSION=base

Let's run the base version to get a baseline runtime.

In [ ]:
%%bash
srun -n 1 ./stencil2d-base.x --nx 128 --ny 128 --nz 64 --num_iter 1024

Take note of the runtime (the last value in the `data` array). This is our sequential baseline. It should be on the order of 0.6 s. As reminder, this time is only the time it takes to execute the loop over the iterations where we apply the diffusion stencil.

## Validation

As a first step, it is always good to make sure that the code is working correctly before proceeding with any sort of parallelization or optimization. We plot the initial and final step to see that the code still produces the same result.

In [ ]:
def read_field_from_file(filename, num_halo=None):
    (rank, nbits, num_halo, nx, ny, nz) = np.fromfile(filename, dtype=np.int32, count=6)
    offset = (3 + rank) * 32 // nbits
    data = np.fromfile(
        filename,
        dtype=np.float32 if nbits == 32 else np.float64,
        count=nz * ny * nx + offset,
    )
    if rank == 3:
        return np.reshape(data[offset:], (nz, ny, nx))
    else:
        return np.reshape(data[offset:], (ny, nx))


def validate_results():
    fig, axs = plt.subplots(1, 2, figsize=(12, 4))

    in_field = read_field_from_file("in_field.dat")
    im1 = axs[0].imshow(
        in_field[in_field.shape[0] // 2, :, :], origin="lower", vmin=-0.1, vmax=1.1
    )
    fig.colorbar(im1, ax=axs[0])
    axs[0].set_title("Initial condition")

    out_field = read_field_from_file("out_field.dat")
    im2 = axs[1].imshow(
        out_field[out_field.shape[0] // 2, :, :], origin="lower", vmin=-0.1, vmax=1.1
    )
    fig.colorbar(im2, ax=axs[1])
    axs[1].set_title("Final result")

    plt.show()

In [ ]:
validate_results()

## Parallelization of the K-Loop
Let's use perftools-lite to generate a performance report for the base version. This will be useful to guide our parallelization approach.

In [ ]:
%%bash
srun -n 1 perf_wrap.sh ./stencil2d-base.x --nx 128 --ny 128 --nz 64 --num_iter 1024

Take a look at the performance report in the file `perf/00000.txt`. Which are the functions which consume most of the runtime? What might be the loops that consume most of the runtime (hot loops)? These code regions should be the primary targets for parallelization with OpenMP.
<div class="alert alert-block alert-info">
<b>Now it's your turn...</b><br>
<b>1.</b> Make a copy of the source code and name it <tt>stencil2d-kparallel.F90</tt>. Compile it (see cell below) and run it (see cell after that). For the time being, the runtime should be the same as the base version. Insert the following block of code immediately at program start to print out the number of available threads.<br>
<b>Fortran</b><br>
<code>!$omp parallel
!$omp master
write(*,*) '#threads = ', omp_get_num_threads()
!$omp end master
!$omp end parallel
</code>
Compile and run your code again. How many threads do we have available? Does the number match the number of cores?
<br>
<b>2.</b> Now use OpenMP to parallelize the <tt>k</tt>-loop. Compile the code again and execute it. Quickly check that the results are still ok (using <tt>validate_results()</tt>, see below)<br>
<b>3.</b> What does the plot tell you? What might be the issue?<br>
<b>4.</b> Irrespective of the result, how fast is your code relative to the base version? What would you have expected?<br>
<b>5.</b> Generate a performance report and check what changed relative to the base version. What are the places that consume most of the runtime now?<br>
<b>6.</b> Rerun for <tt>nz</tt>=1, 24, 25, 48, 64, 480 and inspect the performance profiling report for each of these settings. Look at how the relative distribution of runtime changes. Can you guess what is happening? Why is <tt>nz</tt>=64 not an ideal setting?<br>
</div>

In [ ]:
%%bash
make VERSION=kparallel

In [ ]:
%%bash
srun -n 1 ./stencil2d-kparallel.x --nx 128 --ny 128 --nz 64 --num_iter 1024

In [ ]:
validate_results()

In [ ]:
%%bash
srun -n 1 ./perf_wrap.sh ./stencil2d-kparallel.x --nx 128 --ny 128 --nz 64 --num_iter 1024

<b>ANSWERS</b><p>

3. ...
 

## Variable Scoping

Inside a parallel region, variables can be either *shared* (all threads see the same copy) or *private* (each thread has its own copy). For now, we have not explicitly declared the scope of variables that are being used in the parallel region. If nothing is specified, the compiler assumes per default that all variables are shared. This is dangerous and can lead to very ugly errors. This is the error you've encountered above.

Some guidelines you can follow to decide the scope of variables are the following:
- Loop indices are private.
- Variables that are used (read or written) solely inside the parallel region are private.
- Variables that are only being read inside the parallel region are shared.

<div class="alert alert-block alert-info">
<b>Now it's your turn...</b><br>
<b>7.</b> In order to avoid the default behavior, append the <code>default(none)</code> clause on the line with the <code>!$omp parallel do</code>.Recompile (see below). The compiler will now complain that the scope of several variables has not explicitly been specified. For each of these variables, you have to decide whether they are <code>private(var)</code> or <code>shared(var)</code>.<br>
<b>8.</b> Once your code compiles, run it and validate that the results are still looking correct.<br>
<b>9.</b> Move a single variable (e.g. <tt>nx</tt>) from shared to private, recompile, run and visualize the results. Do you understand what happened?<br>
<b>10.</b> Once you are sure that the variable scoping is correct, run one final time and compare the runtime against the previous version and the baseline. Do you understand why we are faster?<br>
</div>

In [ ]:
%%bash
make VERSION=kparallel

In [ ]:
%%bash
srun -n 1 ./stencil2d-kparallel.x --nx 128 --ny 128 --nz 64 --num_iter 1024

In [ ]:
validate_results()

In [ ]:
%%bash
srun -n 1 ./perf_wrap.sh ./stencil2d-kparallel.x --nx 128 --ny 128 --nz 64 --num_iter 1024

<b>ANSWERS</b><p>

9. ...

10. ...

## Parallelization of the J-Loop
<div class="alert alert-block alert-info">
<b>Now it's your turn...</b><br>
<b>11.</b> Make a copy of the <tt>stencil2d-kparallel.F90</tt> program and name it <tt>stencil2d-jparallel.F90</tt>. This time use OpenMP to parallelize the <tt>j</tt>-loop. Make sure to remove the <tt>k</tt>-loop parallelization. Compile it and run it. How fast is your code? What would you have expected?<br>
<b>12.</b> Again generate a report. What changed? Can you use the report to explain your findings above?<br>
<b>13.</b> Run you code using just 1 thread (<code>OMP_NUM_THREADS=1</code>). How does the runtime compare against the base version? Run the both versions of the code several times to make sure the result is reproducible. What could be the reason for the difference?<br>
<b>14.</b> Is it a good idea to try and parallelize the <tt>i</tt>-loop? Why?<br>
</div>

In [ ]:
%%bash
make VERSION=jparallel

In [ ]:
%%bash
srun -n 1 ./stencil2d-jparallel.x --nx 128 --ny 128 --nz 64 --num_iter 1024

In [ ]:
validate_results()

In [ ]:
%%bash
srun -n 1 ./perf_wrap.sh ./stencil2d-jparallel.x --nx 128 --ny 128 --nz 64 --num_iter 1024

In [ ]:
%%bash
export OMP_NUM_THREADS=1
srun -n 1 ./stencil2d-jparallel.x --nx 128 --ny 128 --nz 64 --num_iter 1024

### Performance analysis

11. ...
12. ...
13. ...
14. ...

## Setting the Number of Threads

We now know that we have 72 cores available on the CPU. We want to investigate what the best number of threads is. While it is possible to increase the number of threads beyond the number of cores, there probably is an optimum.

<div class="alert alert-block alert-info">
<b>Now it's your turn...</b><br>
<b>15.</b> Use the <tt>stencil2d-kparallel.F90</tt> version of the code you developed above and find the optimal number of threads.<br>
<b>16.</b> Do the same thing for the version where you have parallelized the j-loop.
</div>

In [ ]:
%%bash
# The file should still be around, but if you need to recompile, here are the steps:
make VERSION=kparallel

In [ ]:
%%bash
# The file should still be around, but if you need to recompile, here are the steps:
make VERSION=jparallel

In [ ]:
%%bash
nthreads=1
echo "out = [None] * 73" > out.txt
for nthreads in `seq 1 72` ; do
export OMP_NUM_THREADS=$nthreads
  ncores=$nthreads
  if [ $nthreads -gt 72 ] ; then
    ncores=72
  fi
  srun -n 1 -c $ncores ./stencil2d-kparallel.x --nx 128 --ny 128 --nz 64 --num_iter 1024 | sed "s/data =/out[$nthreads] =/g" >> out.txt
done

In [ ]:
exec(open("out.txt", "r").read())
labels = []
times = []
for i in range(len(out)):
    if i == 0:
        continue
    labels.append(i)
    times.append(out[i][:, 5].item())

In [ ]:
x = np.arange(len(labels))  # the label locations
width = 0.35  # the width of the bars
fig = plt.figure()
ax = fig.add_axes([0, 0, 1, 1])
ax.bar(labels, times)
ax.set_xlabel('Number of Cores')
ax.set_ylabel('Runtime')
plt.show()

In [ ]:
%%bash
nthreads=1
echo "out = [None] * 73" > out_j.txt
for nthreads in `seq 1 72` ; do
export OMP_NUM_THREADS=$nthreads
  ncores=$nthreads
  if [ $nthreads -gt 72 ] ; then
    ncores=72
  fi
  srun -n 1 -c $ncores ./stencil2d-jparallel.x --nx 128 --ny 128 --nz 64 --num_iter 1024 | sed "s/data =/out[$nthreads] =/g" >> out_j.txt
done

In [ ]:
exec(open("out_j.txt", "r").read())
labels = []
times = []
for i in range(len(out)):
    if i == 0:
        continue
    labels.append(i)
    times.append(out[i][:, 5].item())

In [ ]:
x = np.arange(len(labels))  # the label locations
width = 0.35  # the width of the bars
fig = plt.figure()
ax = fig.add_axes([0, 0, 1, 1])
ax.bar(labels, times)
ax.set_xlabel('Number of Cores')
ax.set_ylabel('Runtime')
plt.show()

<div class="alert alert-block alert-warning">
The concepts presented up to here are key to understanding how to optimize weather and climate codes. The next set of questions should deepen your understanding of OpenMP and shared memory parallelism in general. The following exercises can be considered a bonus if the time is too short.<br>
The questions are split into two sections: Deeper understanding of OpenMP and playing with minor optimizations in the stencil2d example. They can be solved in either order.
</div>

# Understanding OpenMP

## Diagnostics of our Field
The first task is to understand how to use critical sections, atomics and proper variable scoping to implement a function that calculates the maximal value present in the 3d field. 
<div class="alert alert-block alert-success">
<b>Now it's your turn...</b><br>
<b>B1.</b> Make a copy of the base source code and name it <tt>stencil2d-max.F90</tt>.<br>
Add the function that iterates over all fields and reports the highest value found after every 100th iteration. Report the value to the standard output (<code>std::cout</code> or <code>write(*, *)</code>)<br>
<b>B2.</b> Make a copy of the maximum computing code and name it <tt>stencil2d-parallelmax.F90</tt>.<br>
In it, parallelize the diagnostics. Can you put this into a parallel section or does it need its own? Can you reproduce the same results?
</div>

In [ ]:
%%bash
make VERSION=max

In [ ]:
%%bash
srun -n 1 ./stencil2d-max.x --nx 128 --ny 128 --nz 64 --num_iter 410

In [ ]:
%%bash
make VERSION=parallelmax

In [ ]:
%%bash
srun -n 1 ./stencil2d-parallelmax.x --nx 128 --ny 128 --nz 64 --num_iter 410

## Manual Implementation of the `omp parallel for` Pragma
There is a way to re-implement what a parallel for loop does with the directives `omp parallel`, `omp private`, `omp single`(or similar) and `omp task`. We let the scheduling happen automagically and do not worry about it for now. 
The second task is to write a simple loop that prints the thread number that executes the iteration as well as the iteration number though the full loop. A skeleton code is available in for forloop.F90.
<div class="alert alert-block alert-success">
<b>Now it's your turn...</b><br>
<b>B3.</b> Insert the required pragmas to make sure that more than one thread works on the various pieces of the code
</div>

In [ ]:
%%bash
gfortran -fopenmp -o forloop.x forloop.F90

In [ ]:
%%bash
srun -n 1 ./forloop.x 10

# Minor Optimizations of Stencil2D

## Fusing the Computation Loops

Currently we have the computation in two ij loops. Since we know that writing into a 2d field and reading it again might be additional memory overhead, we would ideally have the computation in just one loop.

There is a way to fuse the computational loops (and have the update on a separate loop). 

The code would have the structure of `stencil2d-fusion.F90`.

<div class="alert alert-block alert-success">
<b>Now it's your turn...</b><br>
<b>B4.</b> Implement the loop bounds and the computation.<br>
<b>B5.</b> Quickly check that the results are still ok (using <tt>validate_results()</tt>, see below)<br>
<b>B6.</b> Use any combination of techniques learned to parallelize this code<br>
<b>B7.</b> Look at the performance in your ideal setting (parallelization with either j or k loop, optimal number of threads)/ How does it compare? How much speedup would you have expected?<br>
    <b>B8.</b> By looking at the perftools-lite report, can you explain the performance?
</div>

In [ ]:
%%bash
make VERSION=fusion

In [ ]:
%%bash
srun -n 1 ./stencil2d-fusion.x --nx 128 --ny 128 --nz 64 --num_iter 1024

In [ ]:
validate_results()

In [ ]:
%%bash
srun -n 1 ./perf_wrap.sh ./stencil2d-fusion.x --nx 128 --ny 128 --nz 64 --num_iter 1024

## Parallelizing the Halo Update

The halo updates are a set of small loops that are all independent. The amount of work done in there independent loops is significantly smaller than the work in the main loop. We try to investigate how to parallelize them efficiently
<div class="alert alert-block alert-success">
<b>Now it's your turn...</b><br>
<b>B9.</b> By looking at the perftools report of previous runs, can you explain why this was a bonus exercise? What is the expected performance gain of this parallelization? <br>
    <b>B10.</b> Make a copy of the source code <tt>stencil2d-kparallel.F90</tt> and name it <tt>stencil2d-kparallel-halo.F90</tt><br>
<b>B11</b> Implement a parallel version of the halo updates with the learnings from before (which loop to parallelize, what number of threads to use, how to scope the variables)<br>
<b>B12</b> Quickly check that the results are still ok (using <tt>validate_results()</tt>, see below)<br>
<b>B13.</b> By looking at the performance report, can you discuss your predictions? Why were they / were they not matched?<br>    
</div>

In [ ]:
%%bash
make VERSION=kparallel-halo

In [ ]:
%%bash
srun -n 1 ./stencil2d-kparallel-halo.x --nx 128 --ny 128 --nz 64 --num_iter 1024

In [ ]:
validate_results()

## Exploring the ideal schedule for our program
We know that in a lot of programs, the loop schedule can have a significant impact on the performance. Since weather and climate codes are usually very nicely structured with similar work loads across iterations (excluding boundary layer computation in the vertical), finding the ideal schedule might only lead to small performance gains. We try to nevertheless understand how different schedules behave and what their performance looks like.

<div class="alert alert-block alert-success">
<b>Now it's your turn...</b><br>
<b>B14.</b> Modify your code stencil2d-jparallel to test these three loop orders: <br>
<code>static(1)
static(ny/p)
dynamic(1)
</code>
   How does the performance compare? Can you explain what these schedules do?<br>
    <b>B15.</b> What would happen if we did the same experiment on the kparallel version?<br>
<b>B16.</b> Can you think of codes where this is more impactful? Why?
</div>


In [ ]:
%%bash
make VERSION=jparallel

In [ ]:
%%bash
srun -n 1 ./stencil2d-jparallel.x --nx 128 --ny 128 --nz 64 --num_iter 1024

In [ ]:
%%bash
make clean